------------------------------
# MiniProject 2: Visualization
------------------------------

Simple version:
- flat surface
- no air resistance
- no rebound

# Description
The program opens an UI window with input fields for the height (H), from where the ball is dropped or shot. Additionally, there are fields for the shooting speed in X, Y and Z coordinates and an input field for the radius of the ball. You also find a preview of the 3D presentation of the ball's trajectory. When you click the "GO" button, the parameters are validated. Incorrect values are rejected.

If the parameters are valid, the ball's trajectory is calculated and plotted with a blue curve leading to the landing size, shown as a green disc. Numeric values for the landing point are displayed in the UI window, together with the calculated flying time. The animation shows an orange ball flying over the trajectory, leaving an orange tail. I tried to align the speed of the animation to the calculated flight time.

All coordinates shown or displayed are for the _center_ of the ball.

You can re-run the animation with the same or new parameters indefinitely. To end it, close the window.

In [10]:
import tkinter as tk
from tkinter import ttk
import numpy as np

from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation

g= 9.81 # m/s²


# calculate time and landing coordinates 
#
def trajectory( H, vx, vy, vz, R ):
  # z(t)= H + vz t - g/2 t² = R    ballistic trajectory
  #       c   bt     at²           a,b,c for Mitternachtsformel
  a= -g / 2.0
  b= vz
  c= H - R
  wurzelterm= b * b - 4 * a * c
  t1= ( -b + np.sqrt( wurzelterm ) ) / ( 2 * a )
  t2= ( -b - np.sqrt( wurzelterm ) ) / ( 2 * a )
  t= max( t1, t2 )                 # choose the longer trajectory

  T= np.linspace( 0, t, 300 )      # 300t values
  X= T * vx                        # constant motion in
  Y= T * vy                        # relation to the floor
  Z= H + T * vz - T*T * g / 2.0    # ballistic trajectory

  return T, X, Y, Z


class BallSimulator:

  def __init__( self, root ):
    self.root= root

    # new UI window
    #
    control= ttk.Frame( root, padding=10 )
    control.pack( side=tk.LEFT, fill=tk.Y )

    # place field descriptions
    #
    ttk.Label( control, text='H [m]').grid(row=0, column=0 )
    ttk.Label( control, text='vx [m/s]').grid(row=1, column=0 )
    ttk.Label( control, text='vy [m/s]').grid(row=2, column=0 )
    ttk.Label( control, text='vz [m/s]').grid(row=3, column=0 )
    ttk.Label( control, text='R [cm]').grid(row=4, column=0 )

    # place input fields
    #
    self.eH= ttk.Entry( control ); self.eH.grid( row=0, column=1 )
    self.eX= ttk.Entry( control ); self.eX.grid( row=1, column=1 )
    self.eY= ttk.Entry( control ); self.eY.grid( row=2, column=1 )
    self.eZ= ttk.Entry( control ); self.eZ.grid( row=3, column=1 )
    self.eR= ttk.Entry( control ); self.eR.grid( row=4, column=1 )

    # pre-fill the input fields with nice parameters
    #
    self.eH.insert( 0, '10' )
    self.eX.insert( 0, '1' )
    self.eY.insert( 0, '1' )
    self.eZ.insert( 0, '10' )
    self.eR.insert( 0, '15' )

    # place the start button
    #
    ttk.Button(control, text='GO', command=self.run).grid(
      row=5, column=0, columnspan=2, pady=10
    )

    self.info= ttk.Label( control, text='' )
    self.info.grid( row=6, column=0, columnspan=2 )

    # place a frame for matplotlib
    #
    fig= Figure( figsize=(6, 5) )
    self.ax = fig.add_subplot( 111, projection='3d' )
    self.ax.set_xlabel( 'x [m]' )
    self.ax.set_ylabel( 'y [m]' )
    self.ax.set_zlabel( 'z [m]' )

    # show empty plot
    #
    self.line, = self.ax.plot([], [], [], lw=2)
    self.point, = self.ax.plot([], [], [], 'ro')

    self.canvas= FigureCanvasTkAgg( fig, master=root )
    self.canvas.get_tk_widget().pack( side=tk.RIGHT, fill=tk.BOTH, expand=True )

    self.ani = None


  def run( self ):
    # read and validate parameters from UI panel
    #
    try:
        H=  float( self.eH.get() )
        vx= float( self.eX.get() )
        vy= float( self.eY.get() )
        vz= float( self.eZ.get() )
        R=  float( self.eR.get() ) / 100    # cm → m
        if R < 0 or H <= R: raise ValueError
    except ValueError:
        self.info.config( text='invalid input' )
        return

    # get time and landing coordinates
    #
    T, X, Y, Z= trajectory( H, vx, vy, vz, R )

    # calculate timing
    #
    skip= 10
    frames= list( range( 0, len(X), skip ) )
    if frames[-1] != len(X) - 1:
      frames.append( len(X) - 1 )

    dt= T[1] - T[0]
    interval_ms= dt * skip * 1000  # ms per step

    # plot trajectory
    #
    self.ax.cla()
    self.ax.set_xlabel( 'x [m]' )
    self.ax.set_ylabel( 'y [m]' )
    self.ax.set_zlabel( 'z [m]' )

    self.ax.plot( X, Y, Z, alpha=0.3 )
    self.ax.scatter( X[-1], Y[-1], Z[-1], color='green', s=50 )

    self.line,  = self.ax.plot( [], [], [], linewidth=2 )
    self.point, = self.ax.plot( [], [], [], color='red', marker='o' )

    # show info
    #
    self.info.config(
      text= f'flying time: {T[-1]:.2f}s,\nlanding point: {X[-1]:.2f}/{Y[-1]:.2f}/{Z[-1]:.2f}m'
    )

    # update animation (step)
    #
    def update( i ):
        self.line.set_data( X[:i], Y[:i] )
        self.line.set_3d_properties( Z[:i] )
        self.point.set_data( [X[i]], [Y[i]] )
        self.point.set_3d_properties( [Z[i]] )
        return self.line, self.point

    frames= list( range( 0, len(X), 10 ) )
    if frames[-1] != len(X)-1:
      frames.append( len(X)-1 )

    self.ani= FuncAnimation(
      self.canvas.figure,
      update,
      frames=frames,
      interval=interval_ms,
      blit=True,
      repeat=False
    )

    self.canvas.draw()

# display UI panel and run UI loop
#
root= tk.Tk()
root.title( 'MiniProject 2' )
BallSimulator( root )
root.mainloop()

